Import

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pickle
import tensorflow as tf
import tensorflowjs as tfjs

Using TensorFlow backend.


Load data

In [2]:
#download from kaggle
#!curl https://www.kaggle.com/crowdflower/first-gop-debate-twitter-sentiment/downloads/Sentiment.csv/2 > Sentiment.csv
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]
# Easy: binary classifier
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)
data.head()

4472
16986


,text,sentiment
1,rt scottwalker didnt catch the full gopdebate ...,Positive
3,rt robgeorge that carly fiorina is trending h...,Positive
4,rt danscavino gopdebate w realdonaldtrump deli...,Positive
5,rt gregabbott_tx tedcruz on my first day i wil...,Positive
6,rt warriorwoman91 i liked her and was happy wh...,Negative


Tokenizer

In [3]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
input_length = X.shape[1]

Prepare Embedding

In [4]:
embed_dim = 128
embedding = Embedding(max_fatures, embed_dim,input_length = input_length)

Create Model

In [5]:
lstm_out = 196
model = Sequential()
model.add(embedding)
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


Train Model

In [6]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

((7188, 28), (7188, 2))
((3541, 28), (3541, 2))
Epoch 1/7
 - 10s - loss: 0.4482 - acc: 0.8144
Epoch 2/7
 - 9s - loss: 0.3239 - acc: 0.8645
Epoch 3/7
 - 9s - loss: 0.2848 - acc: 0.8824
Epoch 4/7
 - 10s - loss: 0.2549 - acc: 0.8932
Epoch 5/7
 - 10s - loss: 0.2335 - acc: 0.9039
Epoch 6/7
 - 10s - loss: 0.2113 - acc: 0.9112
Epoch 7/7
 - 10s - loss: 0.1879 - acc: 0.9225


Evaluate Model

In [7]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.43
acc: 0.84


Test Model

In [14]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#twt = ['We all are great people']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=input_length, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
print(sentiment)
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  204  625    7  148    6   56 1039   56   47    7  148]]
[  9.99512792e-01   4.87158191e-04]
negative


Export Model and Metadata